In [1]:
# Import all the things
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.figure_factory as ff
import waterfall_chart
import altair as alt
from vega_datasets import data as vega_data
import plotly.plotly as py
import plotly.graph_objs as go
from itertools import product
import colorlover as cl


# General settings
pd.options.display.max_rows = 10
pd.options.display.max_seq_items = 200

static_dir = "slides/static/"

In [2]:
%store -r df

Let's go state-by-state.  Which ones flipped?

Sources: 
* https://www.nytimes.com/elections/2016/results/president
* https://www.nationalpopularvote.com/campaign-events-2016



| State | Electoral Votes | Trump Votes | Hillary Votes | Difference |
| ---   | ---             | ---         | ---           | ---        |
| FL    | 29              | 4,617,886   | 4,504,975     | 112911     |
| PA    | 20              | 2,970,733   | 2,926,441     | 44292      |
| OH    | 18              | 2,841,005   | 2,394,164     | 446841     |
| MI    | 16              | 2,279,543   | 2,268,839     | 10704      |
| WI    | 10              | 1,405,284   | 1,382,536     | 22748      |
| IA    | 6               | 800,983     | 653,669       | 147314     |




In [ ]:
def percent_to_float(val):
    return float(val.strip('%'))

def strip_star(val):
    return val.strip('*')

state_df = pd.read_csv('data/state_data.csv', converters={'D Margin 2016':percent_to_float,'D Margin 2012':percent_to_float, 'Margin Shift':percent_to_float, 'State':strip_star})
state_df.head()
df_flipped = state_df[(state_df['D Margin 2016']<0) & (state_df['D Margin 2012']>0)]
df_dem = state_df[(state_df['D Margin 2016']>0) & (state_df['D Margin 2012']>0)]
df_rep = state_df[(state_df['D Margin 2016']<0) & (state_df['D Margin 2012']<0)]


flipped = go.Scatter(
    x = df_flipped['D Margin 2012'],
    y = df_flipped['D Margin 2016'],
    name = 'Flipped States',
    mode = 'markers',
    text=df_flipped.State,
    marker = dict(
        size = 10,
        color = 'yellow',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
dem = go.Scatter(
    x = df_dem['D Margin 2012'],
    y = df_dem['D Margin 2016'],
    name = 'Democratic States',
    mode = 'markers',
    text=df_dem.State,
    marker = dict(
        size = 10,
        color = 'blue',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
rep = go.Scatter(
    x = df_rep['D Margin 2012'],
    y = df_rep['D Margin 2016'],
    name = 'Republican States',
    mode = 'markers',
    text=df_rep.State,
    marker = dict(
        size = 10,
        color = 'red',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)
datarange = np.array([-50, 90])
xy = go.Scatter(
    x=datarange,
    y=datarange,
    mode='lines',
    marker= dict(color='black'),
    name='X=Y'
)
data = [flipped, dem, rep, xy]

layout = dict(title = 'Dem Margin by State, 2012 and 2016',
              yaxis = dict(zeroline = False, title='2016 Dem Margin'),
              xaxis = dict(zeroline = False, title='2012 Dem Margin'),
              width=600,
              height=500,
             )

fig = dict(data=data, layout=layout)

# IPython notebook
plotly.offline.plot(fig, filename='state_scatter')


In [ ]:
state_df['grouping'] = np.where(state_df.State.isin(['Michigan', 'Wisconsin', 'Iowa', 'Pennsylvania', 'Ohio', 'Florida']), 
                             state_df['State'],
                             'Other'
                            )
state_df['2016_winner'] = np.where(state_df['D Margin 2016']>0, 'Democrat Won', 'Republican Won')

In [ ]:
stacked_data = state_df.groupby(['2016_winner', 'grouping'])['Electoral Votes'].sum().unstack()
stacked_data = stacked_data.fillna(0)[['Other', 'Florida', 'Pennsylvania', 'Ohio', 'Michigan', 'Wisconsin', 'Iowa']]
stacked_data.plot(kind='bar', stacked=True)

In [5]:
subset = df[['votes_dem_2008','votes_gop_2008', 'votes_dem_2012', 'votes_gop_2012', 'votes_dem_2016', 'votes_gop_2016','votes_oth_2008','votes_oth_2012','votes_oth_2016', 'state_abbr']][df.state_abbr.isin(['MI', 'WI', 'MN', 'IA', 'PA', 'OH', 'FL'])].groupby('state_abbr').sum()

trace5 = go.Bar(
    x=subset.index,
    y=subset['votes_dem_2008'],
    name='Dem Votes 2008'
)
trace6 = go.Bar(
    x=subset.index,
    y=subset['votes_gop_2008'],
    name='GOP Votes 2008'
)
trace1 = go.Bar(
    x=subset.index,
    y=subset['votes_dem_2012'],
    name='Dem Votes 2012'
)
trace2 = go.Bar(
    x=subset.index,
    y=subset['votes_gop_2012'],
    name='GOP Votes 2012'
)
trace3 = go.Bar(
    x=subset.index,
    y=subset['votes_dem_2016'],
    name='Dem Votes 2016'
)
trace4 = go.Bar(
    x=subset.index,
    y=subset['votes_gop_2016'],
    name='GOP Votes 2016'
)
trace7 = go.Bar(
    x=subset.index,
    y=subset['votes_oth_2008'],
    name='Other Votes 2008'
)
trace8 = go.Bar(
    x=subset.index,
    y=subset['votes_oth_2012'],
    name='Other Votes 2012'
)
trace9 = go.Bar(
    x=subset.index,
    y=subset['votes_oth_2016'],
    name='Other Votes 2016'
)
data=[trace5, trace6, trace1, trace2, trace3, trace4, trace7, trace8, trace9]
layout = go.Layout(barmode='group')
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename=static_dir+'states/grouping.html')



'file:///Users/bwarren/SideProjects/voting/slides/static/states/grouping.html'